<a href="https://colab.research.google.com/github/SonnyDev/llm-apps-langchain/blob/main/Embeddings_et_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Embeddings

Les embeddings offrent des représentations vectorielles denses des mots
et capturent les caractéristiques sémantiques de mots. Ils permettent la mesure de la similarité sémantique entre les mots.
Il existe de nombreux fournisseurs de modèles d’embeddings (OpenAI, Cohere, Hugging Face, etc.) La classe Embeddings de LangChain est conçue pour fournir une interface standard pour tous.

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [ ]:
embeddings[1][:5]

[0.004510450978244393,
 -0.01431200798337721,
 0.00197642011460647,
 -0.015290924838637436,
 -0.02646792231226287]

In [ ]:
# Embed document
from langchain.document_loaders import TextLoader

loader = TextLoader("./langchain.txt", encoding="utf-8")

docs = loader.load()

docs

# Transform documents

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)

texts = text_splitter.split_documents(docs)


for doc in texts:
    embedding = embeddings_model.embed_documents(doc.page_content)
    embeddings.append(embedding)

len(embeddings), len(embeddings[0])

(58, 1536)

In [ ]:
embeddings[0][:5]

[-0.020262643931117454,
 -0.006984279861728337,
 -0.022630838723440946,
 -0.02634143617913019,
 -0.03697932214749123]

In [ ]:
# Embed query

embedded_query = embeddings_model.embed_query("Qu'est-ce qu'une chaine ?")
embedded_query[:5]

[0.004442663852323625,
 -0.016318678663383088,
 -0.010234509710864969,
 -0.01468681042451571,
 -0.013427573699114638]

### 2. Vector Database

In [ ]:
# ChromaDB (!pip install chromadb)

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


In [ ]:
texts[2]

Document(page_content="être utilisés pour générer du texte, répondre à des questions, ou effectuer d'autres tâches liées au langage naturel.", metadata={'source': './langchain.txt', 'start_index': 196})

In [ ]:
chroma_db = Chroma.from_documents(
    documents=texts,
    embedding=embeddings_model,
    persist_directory="vector_db",
    collection_name="lc_chroma_db"
)

In [ ]:
# Similarity search

query = "Qu'est-ce qu'un agent ?"

similar_docs = chroma_db.similarity_search(query)

In [ ]:
similar_docs

[Document(page_content="SQL à l'aide d'agents.", metadata={'source': './langchain.txt', 'start_index': 3730}),
 Document(page_content="SQL à l'aide d'agents.", metadata={'source': './langchain.txt', 'start_index': 0, 'tag': 'agents'}),
 Document(page_content="• Agents : Les agents permettent d'utiliser des modèles de langage comme moteur de raisonnement en prenant des décisions et en effectuant des actions en fonction des instructions et des données", metadata={'source': './langchain.txt', 'start_index': 1232}),
 Document(page_content="• Agents : Les agents permettent d'utiliser des modèles de langage comme moteur de raisonnement en prenant des décisions et en effectuant des actions en fonction des instructions et des données", metadata={'source': './langchain.txt', 'start_index': 1232})]

In [ ]:
# Obtenir tous les documents d'une collection

collection = chroma_db.get()

collection

{'ids': ['0d6d65f5-a35b-11ee-9387-c17a6171887f',
  '0d6d65f6-a35b-11ee-b467-c17a6171887f',
  '0d6d65f7-a35b-11ee-b3a7-c17a6171887f',
  '0d6d65f8-a35b-11ee-a719-c17a6171887f',
  '0d6d65f9-a35b-11ee-8aa7-c17a6171887f',
  '0d6d65fa-a35b-11ee-b8c6-c17a6171887f',
  '0d6d65fb-a35b-11ee-bbb9-c17a6171887f',
  '0d6d65fc-a35b-11ee-bb21-c17a6171887f',
  '0d6d65fd-a35b-11ee-b9a5-c17a6171887f',
  '0d6d65fe-a35b-11ee-a5c6-c17a6171887f',
  '0d6d65ff-a35b-11ee-9729-c17a6171887f',
  '0d6d6600-a35b-11ee-be83-c17a6171887f',
  '0d6d6601-a35b-11ee-aa63-c17a6171887f',
  '0d6d6602-a35b-11ee-b8d6-c17a6171887f',
  '0d6d6603-a35b-11ee-b32d-c17a6171887f',
  '0d6d6604-a35b-11ee-ac63-c17a6171887f',
  '0d6d6605-a35b-11ee-809c-c17a6171887f',
  '0d6d6606-a35b-11ee-abd2-c17a6171887f',
  '0d6d6607-a35b-11ee-a194-c17a6171887f',
  '0d6d6608-a35b-11ee-b361-c17a6171887f',
  '0d6d6609-a35b-11ee-82c4-c17a6171887f',
  '0d6d660a-a35b-11ee-bd39-c17a6171887f',
  '0d6d660b-a35b-11ee-8bb4-c17a6171887f',
  '0d6d660c-a35b-11ee-99e7-

In [ ]:
# Mettre à jour une collection

# assigning custom tag metadata to first document
similar_docs[0].metadata = {
    "tag": "agents"
}

# updating the vector store
chroma_db.update_document(
    document=similar_docs[0],
    document_id=collection['ids'][0]
)

# using the where parameter to filter the collection
collection = chroma_db.get(where={"tag" : "agents"})
collection

{'ids': ['0d6d65f5-a35b-11ee-9387-c17a6171887f',
  '37ebbc89-a340-11ee-b46f-c17a6171887f'],
 'embeddings': None,
 'metadatas': [{'source': './langchain.txt',
   'start_index': 0,
   'tag': 'agents'},
  {'source': './langchain.txt', 'start_index': 0, 'tag': 'agents'}],
 'documents': ["• Mémoire : La mémoire permet de stocker des informations entre les appels d'une chaîne ou d'un agent. Elle peut être utilisée pour conserver des états, des variables ou des résultats intermédiaires",
  "SQL à l'aide d'agents."],
 'uris': None,
 'data': None}